# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 16

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch data
df = pd.read_csv('../data/data_week' + str(gameweek) + '.csv', index_col=0, encoding='latin-1')
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,games played,adjusted points,adjusted points per game,form 5,value,valuePoints metric
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,24,100.0,100.0,0,69140,0,0,-3,3,0.8,0,2,0.5,1.5,0,Shkodran,1.0,2,0,7.5,False,19.8,90,NaN,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,4.0,0,0,Mustafi,0.3,False,NaN,a,1,3,54.0,4,7667,72,31570,195,0.2,0.8,Mustafi,0,1.000000,4.308389,4.308389,NaN,0.828536,1.889353
2,0,0,26,75.0,100.0,0,98745,0,0,-1,1,22.7,0,2,-0.2,0.7,0,Héctor,0.2,6,0,6.9,False,30.8,262,Hamstring injury - 75% chance of playing,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,Bellerín,0.3,False,NaN,d,1,3,15.0,1,34509,502,30240,943,0.0,0.2,Bellerín,2,3.333333,4.147716,1.244315,NaN,0.230429,0.535468
3,2,1,191,100.0,100.0,1,111457,0,0,-3,3,148.9,0,2,1.5,2.5,5,Sead,2.0,14,0,37.3,False,172.6,889,NaN,2019-11-21T14:30:19.207005Z,52,0,0,0,111457.jpg,2.1,0,0,Kolasinac,0.6,False,NaN,a,1,3,53.0,27,45487,339,106342,720,0.4,5.2,Kolasinac,2,12.857143,36.192194,2.814948,3.210629,0.541336,1.234437
4,2,3,106,100.0,100.0,1,154043,0,0,-4,4,110.0,1,2,0.0,1.0,2,Ainsley,0.5,11,0,29.3,False,155.0,580,NaN,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.7,1,0,Maitland-Niles,2.6,False,NaN,a,1,3,33.0,19,544862,228,560581,3143,0.1,4.1,Maitland-Niles,0,7.037037,21.419992,3.043894,3.175771,0.661716,1.419223
5,0,2,217,NaN,NaN,2,39476,0,0,-1,1,30.4,0,2,0.0,1.0,2,Sokratis,0.5,20,1,43.1,False,328.8,1260,NaN,NaN,49,0,0,0,39476.jpg,2.4,0,0,Papastathopoulos,1.6,False,NaN,a,1,3,71.0,33,106569,326,134728,2389,0.1,6.7,Sokratis,4,13.750000,33.146827,2.410678,1.860761,0.491975,1.089033


In [4]:
# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [5]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [7]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [8]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [9]:
metric = 'valuePoints metric'
#metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
411,Patrício,Wolves,52,15.897436,62,69.102762,3.9,4.346787,3.998669,1.906192
280,Krul,Norwich City,45,14.054054,52,53.647562,3.7,3.817230,3.086818,1.799460
341,Gazzaniga,Tottenham,46,10.000000,31,38.569610,3.1,3.856961,3.287341,1.798317
553,Martin,West Ham,40,3.023256,13,10.699151,4.3,3.538950,NaN,1.769475
47,Ryan,Brighton,48,15.853659,65,60.804073,4.1,3.835334,3.291170,1.750582
471,Henderson,Sheffield Utd,47,14.871795,58,56.142565,3.9,3.775103,3.845623,1.741325
168,Schmeichel,Leicester City,54,16.190476,68,65.334958,4.2,4.035394,3.903842,1.736557
494,Ramsdale,Bournemouth,46,15.789474,60,58.646777,3.8,3.714296,2.846363,1.731799
235,de Gea,Manchester Utd,55,16.129032,50,64.296178,3.1,3.986363,2.775315,1.699791


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
297,Lundstram,Sheffield Utd,51,15.918367,78,80.827459,4.9,5.077622,3.952449,2.248409
182,Alexander-Arnold,Liverpool,73,16.190476,68,86.603092,4.2,5.349014,4.742521,1.979759
162,Evans,Leicester City,52,15.957447,75,69.987096,4.7,4.385858,6.694943,1.923326
181,Robertson,Liverpool,71,16.000000,72,81.414655,4.5,5.088416,3.983191,1.909648
185,Matip,Liverpool,52,6.956522,32,30.036645,4.6,4.317768,NaN,1.893466
161,Chilwell,Leicester City,58,12.941176,66,58.033071,5.1,4.484374,4.966420,1.862035
164,Söyüncü,Leicester City,51,15.909091,70,66.659146,4.4,4.190003,5.231467,1.855365
208,Otamendi,Manchester City,50,10.000000,36,41.205933,3.6,4.120593,4.787535,1.842785
405,Boly,Wolves,48,8.095238,34,32.618384,4.2,4.029330,NaN,1.839129


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
215,De Bruyne,Manchester City,102,14.923077,97,96.095088,6.5,6.439362,5.774794,2.016242
171,Maddison,Leicester City,77,15.087719,86,81.532926,5.7,5.403927,7.592412,1.947440
196,Keita,Liverpool,58,3.018868,16,14.064654,5.3,4.658916,11.362441,1.934510
191,Salah,Liverpool,122,12.941176,88,85.677089,6.8,6.620502,8.060977,1.895444
431,Pulisic,Chelsea,75,13.125000,63,66.231155,4.8,5.046183,4.796177,1.842606
192,Mané,Liverpool,123,14.931507,109,95.617439,7.3,6.403737,8.423082,1.825917
287,Cantwell,Norwich City,47,15.789474,60,61.717863,3.8,3.908798,3.622369,1.802994
239,Martial,Manchester Utd,76,10.000000,52,49.518130,5.2,4.951813,4.487797,1.796211
29,Grealish,Aston Villa,60,14.042553,66,60.869827,4.7,4.334669,3.890697,1.769621


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
233,Rashford,Manchester Utd,89,16.031746,101,100.170370,6.3,6.248251,8.399026,2.094418
460,Abraham,Chelsea,79,14.923077,97,87.362222,6.5,5.854170,5.839820,2.082820
166,Vardy,Leicester City,101,16.000000,128,102.916296,8.0,6.432269,9.726321,2.023967
278,Pukki,Norwich City,66,16.153846,84,78.162222,5.2,4.838614,4.746643,1.883428
409,Jiménez,Wolves,75,16.000000,80,81.716296,5.0,5.107269,5.630897,1.864911
313,Ings,Southampton,63,16.153846,84,74.554074,5.2,4.615252,6.140489,1.838760
502,Maupay,Brighton,58,16.000000,64,70.600000,4.0,4.412500,5.026667,1.832191
68,King,Bournemouth,61,11.904762,50,51.462222,4.2,4.322827,7.705919,1.750261
11,Aubameyang,Arsenal,109,16.031746,101,87.062222,6.3,5.430614,7.350845,1.644886


Below we compile a list sorting players based on a given metric (irrespective of position).

In [10]:
#metric = 'adjusted points per game'
metric = 'form 5'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

,web_name,team,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
196,Keita,10,58,3.018868,16,14.064654,5.3,4.658916,11.362441,1.934510
166,Vardy,9,101,16.000000,128,102.916296,8.0,6.432269,9.726321,2.023967
192,Mané,10,123,14.931507,109,95.617439,7.3,6.403737,8.423082,1.825917
233,Rashford,12,89,16.031746,101,100.170370,6.3,6.248251,8.399026,2.094418
191,Salah,10,122,12.941176,88,85.677089,6.8,6.620502,8.060977,1.895444
68,King,3,61,11.904762,50,51.462222,4.2,4.322827,7.705919,1.750261
171,Maddison,9,77,15.087719,86,81.532926,5.7,5.403927,7.592412,1.947440
11,Aubameyang,1,109,16.031746,101,87.062222,6.3,5.430614,7.350845,1.644886
162,Evans,9,52,15.957447,75,69.987096,4.7,4.385858,6.694943,1.923326
